<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Chapter8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
本章でやること：固有表現抽出
つまり文章から人名、組織名といったものを抽出する
まずは簡単な方法を学んで、次にBERTを使ったものを学んでいく
'''

'\n本章でやること：固有表現抽出\nつまり文章から人名、組織名といったものを抽出する\nまずは簡単な方法を学んで、次にBERTを使ったものを学んでいく\n'

In [2]:
# まずはディレクトリ移動
!mkdir chap8
%cd ./chap8


/content/chap8


In [3]:
# 次はライブラリ
# バージョンが教科書のだと古いようなので、模範解答（https://github.com/stockmarkteam/bert-book/blob/master/Chapter6.ipynb）にバージョンを合わせている
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

# 集合に関する数学的な操作のライブラリかな？
# https://qiita.com/anmint/items/37ca0ded5e1d360b51f3
import itertools

import random
import json
from tqdm import tqdm
import numpy as np

# unicode文字列に対していろいろ操作できそうなライブラリ
# https://docs.python.org/ja/3/library/unicodedata.html
import unicodedata

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification
import pytorch_lightning as pl

# 日本語学習済みモデル
MODEL_NAME = 'cl-tohoku/best-base-japanese-whole-word-masking'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 6.7 MB/s 
     |████████████████████████████████| 486 kB 45.0 MB/s 
     |████████████████████████████████| 13.4 MB 47.0 MB/s 
     |████████████████████████████████| 582 kB 45.6 MB/s 
     |████████████████████████████████| 6.6 MB 29.7 MB/s 
     |████████████████████████████████| 120 kB 65.6 MB/s 
     |████████████████████████████████| 880 kB 50.5 MB/s 
     |████████████████████████████████| 419 kB 62.4 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=8b9cf035715c54b2a9d6498533d41a5d039b8bcf5dafb57149c904107e43cf81
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c24c69c8453d894f80b84a8bebf3b8e3a24caa00eab91f4b858b47f350156d70
  Stored in direc

In [4]:
'''
固有表現抽出とは

人名・組織名などの固有名詞
日付などの時間表現
金額などの数値表現

などを抽出する。そして、それが人名なのかなんなのか、カテゴリを判定する

ちなみに、どんなカテゴリが定義されているかは何パターンかあり、問題によって変える
IREX
拡張固有表現階層
などがある
'''

'\n固有表現抽出とは\n\n人名・組織名などの固有名詞\n日付などの時間表現\n金額などの数値表現\n\nなどを抽出する。そして、それが人名なのかなんなのか、カテゴリを判定する\n\nちなみに、どんなカテゴリが定義されているかは何パターンかあり、問題によって変える\nIREX\n拡張固有表現階層\nなどがある\n'

In [5]:
'''
表記揺れについて

固有表現には全角・半角などの違いで、表記揺れが発生する
よって、これらを正規化することで１つに統一する必要がある

正規化には、unicodedata.normalize('NFKC', text) が用いられる
NFKCは正規化のモード
textは正規化対象の文字列

'''
# 匿名関数lambda
normalize = lambda s: unicodedata.normalize("NFKC", s)

# 全角ABCを半角に正規化
print(f'ＡＢＣ　-> {normalize("ＡＢＣ")}')
# 半角ABCを半角に正規化
print(f'ABC　-> {normalize("ABC")}')

# 全角１２３を半角に正規化
print(f'１２３　-> {normalize("１２３")}')
# 半角ABCを半角に正規化
print(f'123　-> {normalize("123")}')

# 全角カタカナを全角に正規化
print(f'アイウ　-> {normalize("アイウ")}')
# 半角カタカナを全角に正規化
print(f'ｱｲｳ　-> {normalize("ｱｲｳ")}')

ＡＢＣ　-> ABC
ABC　-> ABC
１２３　-> 123
123　-> 123
アイウ　-> アイウ
ｱｲｳ　-> アイウ


In [6]:
'''
固有表現抽出方法１
IO法

簡単なアルゴリズムであるが、一般的にはBIO法の方がよく使われる
    IO法でもまあまあいいが、一部対応できない表現があるためBIOが使われる傾向にある

アルゴリズム
１。トークンが固有表現の一部であれば、トークンにタグ「I-(Type)」をつける.  ex) I - (人名)　など
２。トークンが固有表現の一部でないならば、トークンのタグはOとする
3。O以外の同じタグが連続している部分トークン列を連結して、固有表現とする
    トークナイザがトークン分割の際に「##」を付与している場合があるので、これは除外して結合する

I は Inside, O は Outsideから来ている


問題点
同じタグを持つ固有名詞が複数続いた場合は、全てが連結されて一つの固有表現として抽出される
日米　などは日本とアメリカで別々のはずなのに、1個に合体してしまう

'''

'\n固有表現抽出方法１\nIO法\n\n簡単なアルゴリズムであるが、一般的にはBIO法の方がよく使われる\n    IO法でもまあまあいいが、一部対応できない表現があるためBIOが使われる傾向にある\n\nアルゴリズム\n１。トークンが固有表現の一部であれば、トークンにタグ「I-(Type)」をつける.  ex) I - (人名)\u3000など\n２。トークンが固有表現の一部でないならば、トークンのタグはOとする\n3。O以外の同じタグが連続している部分トークン列を連結して、固有表現とする\n    トークナイザがトークン分割の際に「##」を付与している場合があるので、これは除外して結合する\n\nI は Inside, O は Outsideから来ている\n\n\n問題点\n同じタグを持つ固有名詞が複数続いた場合は、全てが連結されて一つの固有表現として抽出される\n日米\u3000などは日本とアメリカで別々のはずなのに、1個に合体してしまう\n\n'

In [7]:
# 固有表現では普段のトークンにタグを付与するので
# それに対応したトークナイザを定義しないといけない

# そのトークナイザの中では、学習・推論に使用する関数を定義
# ここは最悪あんまり理解してなくてもいいらしい

class NER_tokenizer(BertJapaneseTokenizer):

    '''
    文章と、それに含まれる固有表現が与えられたときに、符号化とラベル列の作成を行う関数
    この関数の出力は、BERTモデルに入力できる形になっている。（学習）
    つまり、固有表現に適切なラベルを付与してBERTに入力できるように変換する関数
    これをBERTに入力すると、ラベルがついているため、損失を返してくれる
    これをもとにパラメータを学習する
    '''
    def encode_plus_tagged(self, text, entities, max_length):

        # 手順１：固有表現の前後でtextを分割し、それぞれのラベルをつけておく

        # entities は配列
        # 配列のようそはdict型
        # {'name' : 固有表現, 'span': [文章中の文字列の開始位置index, 終了index+1], 'type' : 固有表現のラベル, 'type_id' : typeに紐づく数字（１：１対応）}
        # sorted :https://note.nkmk.me/python-list-sort-sorted/
        # 単語の文章中での開始位置でソート
        entities = sorted(entities, key=lambda x: x['span'][0])

        # 分割後の文字列を追加する
        splitted = []

        # 次にラベルをつけるのはどこからか？を表す
        position = 0

        for entity in entities:
            start = entitiy['span'][0]
            end = entity['span'][1]

            # 固有表現のラベル
            label = entity['type_id']

            # 固有表現でないものにはラベルOをつける
            splitted.append({'text' : text[position:start], 'label' : 0})

            # 固有表現には対応するラベルをIDで付与
            splitted.append({'text' : text[start:end], 'label' : label})

            # 次にラベルをつけるのはend以降
            position = end

        # 最後余った文字列をラベル0にする
        splitted.append({'text' : text[position:], 'label' : 0})

        # 長さ０の文字列をsplittedから取り除く
        splitted = { s for s in splitted if s['text']}

        # 手順２：分割されたそれぞれの文字列をトークン化し、ラベルづけする

        # トークンを入れる
        tokens = []
        # ラベルを入れる
        labels = []

        for text_splitted in splitted:
            text = text_splitted['text']
            label = text_splitted['label']

            # テキストをトークン化
            tokens_splitted = self.tokenizer(text)
            # 対応するラベルを追加
            labels_splitted = [label] * len(tokens_splitted)

            tokens.extend(tokens_splitted)
            labels.extend(labels_splitted)

        
        # 手順３：符号化を行い、BERTに入力可能にする
        input_ids = self.convert_tokens_to_ids(tokens)

        # 公式ドキュメントが見つからないが
        # prepare_for_model はinput_idsを符号化するらしい
        # 公式これか？？https://huggingface.co/docs/transformers/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.prepare_for_model

        encoding = self.prepare_for_model(
            input_ids,
            max_length=max_length,
            padding='max_length',
            truncation=True
        )


        # 特殊トークン[CLS], [SEP]のラベルを０にする
        labels = [0] + labels[:max_length-2] + [0]

        # [PAD]もラベルを０に
        # [PAD]は文末にあるので、その数だけ最後に追加すればいい
        labels = labels + [0]*(max_length - len(labels))

        encoding['labels'] = labels

        return encoding



